# Chapte 7 - Input-Output Operations
입출력 작업

In [1]:
# 기본 입출력 작업 : text, ndarray, SQL
# pandas 모듈을 활용 : 상동 및 CSV, JSON을 추가적 가능
# PyTable을 사용한 입출력 : HDF5 표준을 활용한 대규모 입출력 작업

<br></br>
## 1 기본 파이썬 입출력
Input-Output Operations

### 01 Pickle 객체 저장
객체를 Disk에 저장하기

In [2]:
import numpy as np
from random import gauss

# 정규분포를 갖는 랜덤데이터를 생성한다
a = [gauss(1.5, 2) for i in range(1000000)] 

In [3]:
%time
import pickle
path = './data/'
with open(path + 'data.pkl', 'wb') as file:
    pickle.dump(a, file)

# 저장된 객체의 용량 9MB확인
# http://www.mireene.com/webimg/linux_tip1.htm  # 리눅스 기본명령어
! ls ./data/*.pkl -l

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.29 µs
-rw-rw-r-- 1 markbaum markbaum 9002006  9월  4 11:29 ./data/data.pkl


In [4]:
# 저장된 객체 불러오기
%time 
with open(path + 'data.pkl', 'rb') as file:
    b = pickle.load(file)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 13.1 µs


In [5]:
# a (생성), b (불러옴) 객체간 동일여부를 확인한다
np.allclose(np.array(a), np.array(b))

True

### 02 Numpy를 활용한 Pickle 객체 저장
by numpy

In [6]:
# 2개 객체를 1개의 파일에 저장하기
# np.array 모듈을 사용해서 속도가 빠르다
# 2배 데이터로, 용량도 2배가 된다
pkl_file = open(path + 'data.pkl', 'wb')  # open file for writing
%time pickle.dump(np.array(a), pkl_file)  # a 객체를 np.array로 변환입력
%time pickle.dump(np.array(a) ** 2, pkl_file) # 동일데이터 제곱을 입력
pkl_file.close()

CPU times: user 28 ms, sys: 4 ms, total: 32 ms
Wall time: 31.4 ms
CPU times: user 24 ms, sys: 4 ms, total: 28 ms
Wall time: 27.8 ms


In [7]:
# 다른 함수들과 같이 실행하면 syntex 오류가 발생한다 (주의!!)
! ls ./data/*.pkl -l

-rw-rw-r-- 1 markbaum markbaum 16000322  9월  4 11:29 ./data/data.pkl


In [8]:
# 2개의 np.array가 저장되었음에도 1개만 출력
pkl_file = open(path + 'data.pkl', 'rb')  # open file for reading
x = pickle.load(pkl_file)
x

array([ 1.70649082,  4.12848061,  3.39685536, ...,  3.6687107 ,
        0.24022276, -0.07239309])

In [9]:
# First In First Out
# pickle.load()를 1번더 실행해야 2번쨰 객체가 출력
# 데이터 구조를 알지 못하면 필요한 자료를 찾기 어렵다
y = pickle.load(pkl_file)
y

array([  2.91211091e+00,   1.70443521e+01,   1.15386263e+01, ...,
         1.34594382e+01,   5.77069768e-02,   5.24075991e-03])

In [10]:
pkl_file.close()

In [11]:
# {'x' : x, 'y' : y} 의 dict 객체를 통해 데이터를 모두 저장
pkl_file = open(path + 'data.pkl', 'wb')  # open file for writing
pickle.dump({'x' : x, 'y' : y}, pkl_file)
pkl_file.close()

In [12]:
# {dict} 을 통해 모든 데이터를 불러오기 가능하다
pkl_file = open(path + 'data.pkl', 'rb')  # open file for writing
data = pickle.load(pkl_file)
pkl_file.close()
for key in data.keys():
    print(key, data[key][:4])

x [ 1.70649082  4.12848061  3.39685536  3.65246595]
y [  2.91211091  17.04435214  11.53862634  13.34050748]


### 02 텍스트 파일 읽고 쓰기
Reading and Writing Text Files

In [13]:
import numpy as np
rows = 5000
a = np.random.standard_normal((rows, 5))  
a.round(4) # 소숫점 4자리서 반올림

array([[ 0.3806, -0.1565, -1.4968, -1.2179, -0.9513],
       [-0.1281,  2.016 , -0.2718, -1.7224,  1.3798],
       [-0.0967, -0.4376, -0.1043, -1.484 ,  0.4281],
       ..., 
       [-0.1536,  1.3108,  1.0434,  0.1261,  0.3519],
       [-1.6063,  0.7243, -1.3272, -1.4951,  1.44  ],
       [-0.3295,  0.7337, -1.8692,  0.4136,  0.6614]])

In [14]:
import pandas as pd
t = pd.date_range(start='2014/1/1', periods=rows, freq='H')
    # set of hourly datetime objects
csv_file = open(path + 'data.csv', 'w')  # open file for writing

In [15]:
header = 'date,no1,no2,no3,no4,no5\n'
csv_file.write(header)

25

In [16]:
# 랜덤한 숫자와 날짜-시간 정보를 합쳐서 1줄씩 기록
for t_, (no1, no2, no3, no4, no5) in zip(t, a):
    s = '%s,%f,%f,%f,%f,%f\n' % (t_, no1, no2, no3, no4, no5)
    csv_file.write(s)
csv_file.close()

In [17]:
# data.csv 저장한 파일 불러오기
csv_file = open(path + 'data.csv', 'r') 
for i in range(5): # 전체 line 중 5줄만 읽기
    print(csv_file.readline(), end='')

date,no1,no2,no3,no4,no5
2014-01-01 00:00:00,0.380571,-0.156497,-1.496833,-1.217939,-0.951262
2014-01-01 01:00:00,-0.128092,2.015992,-0.271780,-1.722366,1.379821
2014-01-01 02:00:00,-0.096663,-0.437576,-0.104265,-1.483974,0.428121
2014-01-01 03:00:00,0.175604,-0.748875,0.454243,1.755248,-1.268272


In [18]:
csv_file = open(path + 'data.csv', 'r')
content = csv_file.readlines()
for line in content[:5]:
    print(line, end='')

date,no1,no2,no3,no4,no5
2014-01-01 00:00:00,0.380571,-0.156497,-1.496833,-1.217939,-0.951262
2014-01-01 01:00:00,-0.128092,2.015992,-0.271780,-1.722366,1.379821
2014-01-01 02:00:00,-0.096663,-0.437576,-0.104265,-1.483974,0.428121
2014-01-01 03:00:00,0.175604,-0.748875,0.454243,1.755248,-1.268272


In [19]:
csv_file.close()

### 03 SQL 데이터베이스
sqlite3

In [20]:
# DB와 연결을 설정
import sqlite3
con = sqlite3.connect(path + 'number.db')

# Query 문 지정 및 실행
query = 'Create Table numbs (Date date, No1 real, No2 real)'
con.execute(query)   # execute : 명령입력
con.commit()         # commit  : 입력시행 (범하다)

In [21]:
# input 01 : 테이블에 데이터 추가하기 : 1 줄 추가
import datetime
con.execute('Insert Into numbs  values(?,?,?)', (datetime.datetime.now(), 0.12, 7.3))

In [22]:
# input 02 : 10,000개 데이터를 생성후 입력하기
data = np.random.standard_normal((10000, 2)).round(5)
print(data.shape)
for row in data:
    con.execute('INSERT INTO numbs VALUES(?, ?, ?)',
                (datetime.datetime.now(), row[0], row[1]))
con.commit()

(10000, 2)


In [23]:
# load 01 : 저장한 데이터 불러오기
con.execute('SELECT * FROM numbs').fetchmany(4)

[('2017-09-04 11:29:29.815855', 0.12, 7.3),
 ('2017-09-04 11:29:29.823753', 1.21111, 0.86575),
 ('2017-09-04 11:29:29.823826', 0.31321, -1.06475),
 ('2017-09-04 11:29:29.823840', -1.77909, -1.54526)]

In [24]:
# load 02 : 한번에 특정 column 만 호출
pointer = con.execute('SELECT   Date, No2   FROM numbs')
for i in range(3):
    print(pointer.fetchone())

('2017-09-04 11:29:29.815855', 7.3)
('2017-09-04 11:29:29.823753', 0.86575)
('2017-09-04 11:29:29.823826', -1.06475)


In [25]:
con.close()
!rm ./data/number.db

### 03 Numpy 활용, 자료형 정의하기
numpy 의 dtype를 지정하면 Sqlite의 자료형 변환의 수고를 덜어준다

In [26]:
# 필드별 데이터타입 정의하기
import numpy as np
# 날짜데이터로 기준 index 생성하기
dtimes = np.arange('2015-01-01 10:00:00', '2021-12-31 22:00:00',
                  dtype='datetime64[m]')  # 분 단위로 index를 생성 (minute intervals)
len(dtimes)

3681360

In [27]:
np.zeros([4,4])

array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [28]:
d_ty = np.dtype([('Date', 'datetime64[m]'), ('No1', 'f'), ('No2', 'f')])
# zero array 생성 (datetime의 0을 입력하니까, 모든 데이터가 1970-1-1로 치환되어 입력)
# 전체 3컬럼 중, 첫번쨰 컬럼에 내용이 삽입
data = np.zeros(len(dtimes), dtype = d_ty); data 

array([('1970-01-01T00:00',  0.,  0.), ('1970-01-01T00:00',  0.,  0.),
       ('1970-01-01T00:00',  0.,  0.), ..., ('1970-01-01T00:00',  0.,  0.),
       ('1970-01-01T00:00',  0.,  0.), ('1970-01-01T00:00',  0.,  0.)],
      dtype=[('Date', '<M8[m]'), ('No1', '<f4'), ('No2', '<f4')])

In [29]:
# zero 데이터 영역에 dtime 객체로 치환
data['Date'] = dtimes; data

array([('2015-01-01T10:00',  0.,  0.), ('2015-01-01T10:01',  0.,  0.),
       ('2015-01-01T10:02',  0.,  0.), ..., ('2021-12-31T21:57',  0.,  0.),
       ('2021-12-31T21:58',  0.,  0.), ('2021-12-31T21:59',  0.,  0.)],
      dtype=[('Date', '<M8[m]'), ('No1', '<f4'), ('No2', '<f4')])

In [30]:
# 기타 column 데이터 입력하기
a = np.random.standard_normal((len(dtimes), 2)).round(5)
data['No1'] = a[:, 0] # 생성한 난수의 0 컬럼을 No1에 입력
data['No2'] = a[:, 1] # 생성한 난수의 1 컬럼을 No1에 입력
data

array([('2015-01-01T10:00', -0.99307001,  0.60544002),
       ('2015-01-01T10:01',  0.59921002, -0.22319999),
       ('2015-01-01T10:02', -2.08380008, -1.64101005), ...,
       ('2021-12-31T21:57', -0.46145999,  1.37346995),
       ('2021-12-31T21:58', -0.91083997,  1.70670998),
       ('2021-12-31T21:59', -0.1596    ,  0.22825   )],
      dtype=[('Date', '<M8[m]'), ('No1', '<f4'), ('No2', '<f4')])

In [43]:
# 59Mb 임에도 속도가 빠르다
%time np.save(path + 'array', data)  # array .npy 로 저장된다
!ls ./data/*.npy -l

CPU times: user 4 ms, sys: 44 ms, total: 48 ms
Wall time: 45.9 ms
-rw-rw-r-- 1 markbaum markbaum 58901888  9월  4 11:32 ./data/array.npy


In [44]:
# numpy 객체 불러오기
%time np.load(path + 'array.npy')

CPU times: user 0 ns, sys: 16 ms, total: 16 ms
Wall time: 14.7 ms


array([('2015-01-01T10:00', -0.99307001,  0.60544002),
       ('2015-01-01T10:01',  0.59921002, -0.22319999),
       ('2015-01-01T10:02', -2.08380008, -1.64101005), ...,
       ('2021-12-31T21:57', -0.46145999,  1.37346995),
       ('2021-12-31T21:58', -0.91083997,  1.70670998),
       ('2021-12-31T21:59', -0.1596    ,  0.22825   )],
      dtype=[('Date', '<M8[m]'), ('No1', '<f4'), ('No2', '<f4')])

In [46]:
# 대용량 데이터 저장 및 불러오기
# 거의 시스템 속도만큼 즉각 저장이 가능하다 
data = np.random.standard_normal((1000, 6000))
%time np.save(path + 'array_big', data)
!ls ./data/*.npy -l

CPU times: user 4 ms, sys: 20 ms, total: 24 ms
Wall time: 22.1 ms
-rw-rw-r-- 1 markbaum markbaum 58901888  9월  4 11:32 ./data/array.npy
-rw-rw-r-- 1 markbaum markbaum 48000080  9월  4 11:36 ./data/array_big.npy


In [47]:
! rm ./data/*.npy
# 결론
# pickle 및 SQL 보다 더 속도가 빠른반면, SQL의 기능을 활용할 수 없다
# SQL기능을 보완가능한 Pandas (Pytable 라이브러리)를 활용하면 유용하다

<br></br>
## 2 pandas 를 활용한 사용자 입출력
pandas 라이브러리